# Traveling Salesman Problem
* 旅行商问题：一辆车的旅行
* 从北京出发，经过其他城市一次，回到北京，总距离最小

In [29]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
class tsp(object):
    def __init__(self, city_names=None,num_vehicles=1):
        self.num_vehicles = num_vehicles
        # 获取城市的名称，经纬度
        self.df = pd.read_excel('./cities.xlsx')
        self.all_city = self.df['name'].values
        
        if city_names is not None:
            # 指定某些城市的旅行商问题
            self.city_names = city_names
            # 如果不筛选df，下标会不对应 北京0 天津1 上海2
            # 筛选出来有city_names的行和列
            self.df = self.df[self.df['name'].isin(city_names)]
        else:
            self.city_names = self.all_city
        
    # 设置数据，初始化data，得到data字典，记录distance_matrix，num_vehicles， depot等数据
    def create_data_model(self):
        temp = pd.read_excel('./distance.xlsx', index_col=0)
        # 按照city_names进行筛选 两点之间的距离
        temp = temp[(temp.index.isin(self.city_names))][self.city_names]
        
        data = {}
        # 设置3个维度： 邻接矩阵（城市之间距离），车的数量，起始点
        data['distance_matrix'] = temp.values / 1000 # 采用公里计算
        # data['num_vehicles'] = 4
        data['depot'] = 0 #起始点的ID，从北京出发
#         print('distance_matrix=\n', data['distance_matrix'])
        return data

    # 输出结果，返回路径(这里为index list，比如[0, 1, 2]) 以及总距离
    def print_solution(self, manager, routing, solution):
        # 记录每辆车的总里程
        distance_list = []
        # 记录每辆车的路径
        route_list = []
        # loss function = cost + 惩罚项
        print('总Cost: {} 公里'.format(solution.ObjectiveValue()))
        # 遍历每辆车
        for vehicle_id in range(self.num_vehicles):
            # 初始化距离为0
            route_distance = 0
            # 针对vehicle_id车辆进行规划
            route = []
            # 从vehicle_id的起始节点出发
            index = routing.Start(vehicle_id)
            plan_output = '车辆的路径:\n'
            route_distance = 0 
            # 判断是否达到终点
            while not routing.IsEnd(index):
                # 显示输出的路径规划
                #print(manager.IndexToNode(index))
                #plan_output += ' {} ->'.format(self.city_names[manager.IndexToNode(index)])
                index_show = manager.IndexToNode(index)
                route.append(index_show)
                previous_index = index
                # 得到下一个城市
                index = solution.Value(routing.NextVar(index))
                # 计算距离 => 添加到总距离中
                route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            # 添加到route_list
            route_list.append(route)
            distance_list.append(route_distance)
                
                
        # 输出最后一个城市
        # plan_output += self.city_names[manager.IndexToNode(index)]
        # print(plan_output)
        # print('{}公里'.format(route_distance))
        return route_list,distance_list

    def work(self):
        # 初始化数据
        data = self.create_data_model()

        # 创建路线管理，tsp_size (邻接矩阵表的大小), num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               self.num_vehicles, data['depot'])

        # 创建路径规划模型 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # 计算两点之间的距离
        def distance_callback(from_index, to_index):
            # 从Index => NodeIndex的转换
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            # 从distance_matrix中的起点、终点 找到距离
            return data['distance_matrix'][from_node][to_node]

        # 创建距离回调函数，用于计算两个节点之间的距离
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        # 对所有的车辆，采用相同的distance callback的计算
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        
        # 添加距离约束 
        dimension_name = 'Distance' 
        routing.AddDimension( 
            transit_callback_index, 
            0, # no slack 
            10000, # 车辆最大行驶距离 
            True, # start cumul to zero 
            dimension_name) 
        distance_dimension = routing.GetDimensionOrDie(dimension_name) 
        # 尽量减少车辆的最大距离 
        distance_dimension.SetGlobalSpanCostCoefficient(100) 
        
        
        # 设置启发式方法（距离成本最短 PATH_CHEAPEST_ARC）
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        # 初始化的解
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
        # 在初始解的基础上，进一步优化（通过邻居调整）
        search_parameters.local_search_metaheuristic = (
            routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
        search_parameters.time_limit.seconds = 5

        #print(routing)
        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出结果
        if solution:
            route,route_distance = self.print_solution(manager, routing, solution)
        else:
            route,route_distance = None,None
            
        return route,route_distance

In [30]:
if __name__ == '__main__':
    # 指定旅行的城市
    city_names = ['北京', '天津', '上海', '南京']
    # 创建一个tsp类的实例
    # model = tsp(city_names=city_names)
    model = tsp(num_vehicles=4)
    route,route_distance = model.work()
# 19799 33个城市的旅行规划
# 19383 33个城市的旅行规划

总Cost: 736631 公里


In [31]:
route

[[0, 7, 18, 19, 17, 22, 6],
 [0, 21],
 [0, 5, 20, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 13, 16],
 [0, 14, 28, 29, 10, 9, 8, 15, 1, 4, 2, 3]]

In [32]:
route_distance

[6341, 7096, 6749, 6845]

In [33]:
7096*100+(6341+7096+6749+6845)

736631